<a href="https://colab.research.google.com/github/jbui009/big-data-challenge/blob/main/Jewelryipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubun

In [2]:
from sqlalchemy import create_engine
import pandas as pd


In [3]:
from sqlalchemy import create_engine
import pandas as pd


In [4]:
 !wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-01 00:47:42--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-03-01 00:47:42 (10.7 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("JewelryCloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [6]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DateType

schema = [StructField("marketplace", StringType(), True),
          StructField("customer_id", IntegerType(), True),
          StructField("review_id", StringType(), True),
          StructField("product_id", StringType(), True),
          StructField("product_parent", IntegerType(), True),
          StructField("product_title", StringType(), True),
          StructField("product_category", StringType(), True),
          StructField("star_rating", IntegerType(), True),
          StructField("helpful_votes", IntegerType(), True),
          StructField("total_votes", IntegerType(), True),
          StructField("vine", StringType(), True),
          StructField("verified_purchase", StringType(), True),
          StructField("review_headline", StringType(), True),
          StructField("review_body", StringType(), True),
          StructField("review_date", DateType(), True),]

final=StructType(fields=schema)


In [7]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Jewelry_v1_00.tsv.gz"), sep="\t", header=True)

df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...| 2015-08-31|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...|         Jewelry|          5|    

In [8]:
print(df.count())
df =df.dropna();
print(df.count())

1767753
1767394


In [9]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [10]:
customers_df = df.select("customer_id").groupBy("customer_id").count().withColumnRenamed("count","customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   22279960|             1|
|   51290010|             1|
|    1354679|             1|
|   44662810|             3|
|    7763633|             1|
|    2994816|             1|
|    5884899|             3|
|   47822294|             1|
|   13801937|             1|
|    5263722|             2|
|     216438|             1|
|    7199609|             1|
|   23224071|             1|
|   21118239|            10|
|   26246534|             1|
|     829118|             1|
|   44517006|             1|
|   24325719|             1|
|    1716767|             2|
|   10600983|             1|
+-----------+--------------+
only showing top 20 rows



In [11]:
customers_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_count: long (nullable = false)



In [12]:
from pyspark.sql import Row
products_df = df.dropDuplicates((["product_id"])).select("product_id", "product_title")

products_df.show(truncate=False)

+----------+------------------------------------------------------------------------------+
|product_id|product_title                                                                 |
+----------+------------------------------------------------------------------------------+
|1438809484|Twilight New Moon Book Bracelet Edward with Bella                             |
|B00006C7GQ|Coral and Turquoise Jewelry Bracelets                                         |
|B00006JMZ8|Delicate Gem Necklace - Citrine                                               |
|B00007G9BS|14K Cultured (Akoya) Pearl Leverback Earrings                                 |
|B00009EOU4|14K White Gold Diamond Initial (1/10 Carat) C Pendant w/Chain                 |
|B00009MEOI|Multi-Colored Jade Ring Set Size 8                                            |
|B0000ATBZZ|Akoya Cultured Pearl 6.5-7.0mm Earrings                                       |
|B0000AWSIN|Platinum 4mm Comfort Fit Milgrain Plain Wedding Band                

In [13]:
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [14]:
review_id_df = df.select("review_id", "customer_id", "product_id", "product_parent", "review_date")
review_id_df.show(truncate=False)

+--------------+-----------+----------+--------------+-----------+
|review_id     |customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R135Q3VZ4DQN5N|50423057   |B00JWXFDMG|657335467     |2015-08-31 |
|R2N0QQ6R4T7YRY|11262325   |B00W5T1H9W|26030170      |2015-08-31 |
|R3N5JE5Y4T6W5M|27541121   |B00M2L6KFY|697845240     |2015-08-31 |
|R2I150CX5IVY9Q|5350721    |B0006SW2WU|569859289     |2015-08-31 |
|R1RM9ICOOA9MQ3|24484424   |B009YPDW70|332947422     |2015-08-31 |
|R2J2KMDL10UMSH|457767     |B00QGA8XZM|424804582     |2015-08-31 |
|R3R9ZUFA4TB4FQ|31009358   |B00L0N0E8E|589441783     |2015-08-31 |
|R3UQ8VAQN7R6WL|7034628    |B00DMWQKWA|517428445     |2015-08-31 |
|R1FXZ69C01JNQM|34608636   |B00THLHAGA|320313750     |2015-08-31 |
|RY36LB4OW0FFS |44972687   |B00UXSEOHY|589365488     |2015-08-31 |
|R6PKS54PC6ZY2 |16684508   |B00XGRYGVS|217045013     |2015-08-31 |
|R1LYSAH0IECVWB|23493178   |B00AJ2VU8A|972853757     |2015-08-

In [15]:
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- review_date: string (nullable = true)



In [16]:
from pyspark.sql.functions import to_date
review_id_df = review_id_df.withColumnRenamed("review_date","date_str")
review_id_df = review_id_df.withColumn("review_date", to_date(review_id_df["date_str"])).select("review_id","customer_id","product_id","product_parent","review_date")
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- review_date: date (nullable = true)



In [17]:
vine_table_df = df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")
vine_table_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+
|review_id     |star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R135Q3VZ4DQN5N|5          |0            |0          |N   |
|R2N0QQ6R4T7YRY|5          |0            |0          |N   |
|R3N5JE5Y4T6W5M|5          |0            |0          |N   |
|R2I150CX5IVY9Q|5          |0            |0          |N   |
|R1RM9ICOOA9MQ3|5          |0            |0          |N   |
|R2J2KMDL10UMSH|5          |0            |0          |N   |
|R3R9ZUFA4TB4FQ|5          |0            |0          |N   |
|R3UQ8VAQN7R6WL|5          |0            |0          |N   |
|R1FXZ69C01JNQM|5          |0            |0          |N   |
|RY36LB4OW0FFS |5          |0            |0          |N   |
|R6PKS54PC6ZY2 |1          |4            |4          |N   |
|R1LYSAH0IECVWB|5          |0            |0          |N   |
|R1UN0IZI3XQ3E2|5          |2            |3          |N   |
|RIKDXD8QGL99S |5          |1           

In [18]:
vine_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)



In [22]:
# Protect password
from getpass import getpass
password = getpass('password')

password··········


In [23]:
mode = "append"
jdbc_url="jdbc:postgresql://mydbinstance.ccn4uoh5hvud.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
configuration = {"user": "postgres", 
          "password": password, 
          "driver": "org.postgresql.Driver"}

In [24]:
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=configuration)

In [25]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=configuration)

In [27]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=configuration)

In [32]:
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=configuration)